In [1]:

import asyncio
import aiohttp
import logging
import json
import datetime as dt
import pandas as pd
import requests
import time
from ohlc import AmberData
from datetime import datetime
import numpy as np
def create_changePCT(df, shift, column, time):
    df[column+"Chg"+time] = (futures_candle_df[column]-shift[column])
    df[column+"Chg%"+time] = df[column+"Chg"+time]/shift[column]
    return df
    

In [2]:
data_start = datetime.now()
amberdata = AmberData()
now = datetime.now()
start = dt.datetime(2018, 1, 1)
end = dt.datetime.combine(dt.date.today(), dt.time.min)

# spot_candles = amberdata.get_spot_candles(pair='btc_usdt', exchange='binance', interval='hours', start=start, end=end)
# spot_candle_df = pd.DataFrame(spot_candles)
# print(spot_candle_df)
ticker = "BTCUSDT"
exchange_in = 'binance'
interval_in = 'hours'
shift_val = 24
futures_candles = amberdata.get_futures_candles(instrument=ticker, exchange=exchange_in, interval=interval_in, start=start, end=end)
futures_candle_df = pd.DataFrame(futures_candles)
print(futures_candle_df)
end = datetime.now()
print('raw data', end-now)
#futures_candle_df['close_forward24'] = futures_candle_df['close'].shift(-1)
futures_candle_df['volume_24HR'] = futures_candle_df['volume'].rolling(24).sum()
# futures_candle_df['volume_24HR$'] = futures_candle_df['volume_24HR'].rolling(24).sum()*futures_candle_df['close']
# futures_candle_df['volume_$'] = futures_candle_df['volume'].rolling(24).sum()*futures_candle_df['close']



futures_candle_df['volume_$'] = futures_candle_df['volume']*futures_candle_df['close']
futures_candle_df['volume_24HR$'] = futures_candle_df['volume_$'].rolling(24).sum()

      exchange      timestamp      open      high       low     close  \
0      binance  1567962000000  10000.00  10000.00  10000.00  10000.00   
1      binance  1567965600000  10000.00  10000.00  10000.00  10000.00   
2      binance  1567969200000  10344.77  10357.53  10337.43  10340.12   
3      binance  1567972800000  10340.12  10368.64  10334.54  10351.42   
4      binance  1567976400000  10351.42  10391.90  10324.77  10391.90   
...        ...            ...       ...       ...       ...       ...   
30282  binance  1676977200000  24699.60  24756.40  24555.00  24572.60   
30283  binance  1676980800000  24572.60  24663.00  24474.80  24634.10   
30284  binance  1676984400000  24634.10  24699.00  24505.60  24568.60   
30285  binance  1676988000000  24568.60  24786.60  24550.20  24686.40   
30286  binance  1676991600000  24686.30  24724.60  24388.00  24554.00   

          volume instrument                  dt  
0          0.002    BTCUSDT 2019-09-08 13:00:00  
1          0.000    BTC

In [3]:
shift_df = futures_candle_df.copy(deep=True).shift(shift_val)
#24hr raw feature changes
futures_candle_df = create_changePCT(futures_candle_df, shift_df, "close", "24HR")
futures_candle_df= create_changePCT(futures_candle_df, shift_df, "open", "24HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df, "high", "24HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df, "low", "24HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df, 'volume_24HR', "24HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df, 'volume_24HR$', "24HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df, 'volume', "24HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df, 'volume_$', "24HR")
#1hr raw feature changes

shift_df1hr = futures_candle_df.copy(deep=True).shift(1)
futures_candle_df = create_changePCT(futures_candle_df, shift_df1hr, "close", "1HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df1hr, "open", "1HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df1hr, "high", "1HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df1hr, "low", "1HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df1hr, 'volume_24HR', "1HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df1hr, 'volume_24HR$', "1HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df, 'volume', "1HR")
futures_candle_df = create_changePCT(futures_candle_df, shift_df, 'volume_$', "1HR")

In [4]:
#for prediction 24hr ahead
futures_candle_df['closeChg%_forward24HR'] = futures_candle_df['closeChg%24HR'].shift(-24)
futures_candle_df['closeChg%_forward1HR'] = futures_candle_df['closeChg%1HR'].shift(-1)
#used parkinson and garman-klass
futures_candle_df['hl_log_sqr'] = np.log(futures_candle_df["high"]/futures_candle_df["low"])**2
#used in garman-klass
futures_candle_df['co_log_sqr'] = np.log(futures_candle_df["close"]/futures_candle_df["open"])**2
#used in rogers-satchell
futures_candle_df['hc_log'] = np.log(futures_candle_df["high"]/futures_candle_df["close"])
futures_candle_df['ho_log'] = np.log(futures_candle_df["high"]/futures_candle_df["open"])
futures_candle_df['lc_log'] = np.log(futures_candle_df["low"]/futures_candle_df["close"])
futures_candle_df['lo_log'] = np.log(futures_candle_df["low"]/futures_candle_df["open"])

In [5]:
day = '3D'
futures_candle_df['close_ewm'+day] = futures_candle_df['close'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['open_ewm'+day] = futures_candle_df['open'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['high_ewm'+day] = futures_candle_df['high'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['low_ewm'+day] = futures_candle_df['low'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['hl_log_sqr_ewm'+day] = futures_candle_df['hl_log_sqr'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['co_log_sqr_ewm'+day] = futures_candle_df['co_log_sqr'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['hc_log_ewm'+day] = futures_candle_df['hc_log'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['ho_log_ewm'+day] = futures_candle_df['ho_log'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['lc_log_ewm'+day] = futures_candle_df['lc_log'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['lo_log_ewm'+day] = futures_candle_df['lo_log'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_ewm'+day] = futures_candle_df['volume'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_24HR_ewm'+day] = futures_candle_df['volume_24HR'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_$_ewm'+day] = futures_candle_df['volume_$'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_24HR$_ewm'+day] = futures_candle_df['volume_24HR$'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()

day = '7D'
futures_candle_df['close_ewm'+day] = futures_candle_df['close'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['open_ewm'+day] = futures_candle_df['open'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['high_ewm'+day] = futures_candle_df['high'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['low_ewm'+day] = futures_candle_df['low'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['hl_log_sqr_ewm'+day] = futures_candle_df['hl_log_sqr'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['co_log_sqr_ewm'+day] = futures_candle_df['co_log_sqr'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['hc_log_ewm'+day] = futures_candle_df['hc_log'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['ho_log_ewm'+day] = futures_candle_df['ho_log'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['lc_log_ewm'+day] = futures_candle_df['lc_log'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['lo_log_ewm'+day] = futures_candle_df['lo_log'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_ewm'+day] = futures_candle_df['volume'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_24HR_ewm'+day] = futures_candle_df['volume_24HR'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_$_ewm'+day] = futures_candle_df['volume_$'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_24HR$_ewm'+day] = futures_candle_df['volume_24HR$'].ewm(halflife='3 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()



day = '21D'
futures_candle_df['close_ewm'+day] = futures_candle_df['close'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['open_ewm'+day] = futures_candle_df['open'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['high_ewm'+day] = futures_candle_df['high'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['low_ewm'+day] = futures_candle_df['low'].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['hl_log_sqr_ewm'+day] = futures_candle_df['hl_log_sqr'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['co_log_sqr_ewm'+day] = futures_candle_df['co_log_sqr'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['hc_log_ewm'+day] = futures_candle_df['hc_log'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['ho_log_ewm'+day] = futures_candle_df['ho_log'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['lc_log_ewm'+day] = futures_candle_df['lc_log'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['lo_log_ewm'+day] = futures_candle_df['lo_log'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_ewm'+day] = futures_candle_df['volume'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_24HR_ewm'+day] = futures_candle_df['volume_24HR'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_$_ewm'+day] = futures_candle_df['volume_$'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
futures_candle_df['volume_24HR$_ewm'+day] = futures_candle_df['volume_24HR$'].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()

In [6]:
#%% make ewm % changes
shifts = [1, 24]
for col in futures_candle_df.columns:
    if "_ewm" in col:
        for s in shifts:
            shift_df_temp = futures_candle_df.copy(deep=True).shift(s)
            futures_candle_df = create_changePCT(futures_candle_df, shift_df_temp, col, str(s)+"HR")
            
            
futures_candle_df.dropna(inplace=True)
futures_candle_df.reset_index(drop = True, inplace=True)  

/tmp/ipykernel_185607/1640958538.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column+"Chg%"+time] = df[column+"Chg"+time]/shift[column]
/tmp/ipykernel_185607/1640958538.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column+"Chg"+time] = (futures_candle_df[column]-shift[column])
/tmp/ipykernel_185607/1640958538.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [7]:
vol_windows = [1*24,3*24,5*24,7*24, 15*24, 30*24, 60*24]
for x in vol_windows:
    #make close2close
    futures_candle_df['vol_c2c_'+str(x/24)+"D"] = (futures_candle_df['closeChg%24HR'].rolling(x).std()*((365)**.5))*100

    
    
    #make parkinsonConsole 9
    futures_candle_df['vol_park_'+str(x/24)+"D"] = (futures_candle_df['hl_log_sqr'].rolling(x).sum()*(1.0 / (4.0*(x)* np.log(2.0))))**.5
    futures_candle_df['vol_park_'+str(x/24)+"D"] = (futures_candle_df['vol_park_'+str(x/24)+"D"]*((365*24)**.5))*100
    
    
    
    #make garman-klass
    mult_one= 1/(2*x)
    mult_two = (2*np.log(2)-1)/x
    futures_candle_df['vol_garman_'+str(x/24)+"D"] = futures_candle_df['hl_log_sqr']-mult_two*futures_candle_df['co_log_sqr']
    futures_candle_df['vol_garman_'+str(x/24)+"D"] = (mult_one*futures_candle_df['vol_garman_'+str(x/24)+"D"].rolling(x).sum())**.5
    futures_candle_df['vol_garman_'+str(x/24)+"D"] = (futures_candle_df['vol_garman_'+str(x/24)+"D"]*((365*24)**.5))*100
    #rogers-stachell
    mult_one = 1/x
    futures_candle_df['vol_rogers_'+str(x/24)+"D"] = (futures_candle_df['hc_log']*futures_candle_df['ho_log'])+ (futures_candle_df['lc_log']*futures_candle_df['lo_log'])
    futures_candle_df['vol_rogers_'+str(x/24)+"D"] = (mult_one*futures_candle_df['vol_rogers_'+str(x/24)+"D"].rolling(x).sum())**.5
    futures_candle_df['vol_rogers_'+str(x/24)+"D"] = (futures_candle_df['vol_rogers_'+str(x/24)+"D"]*((365*24)**.5))*100


    #make EWM base don halflives
    futures_candle_df['vol_c2c_'+str(x/24)+"D"+"_ewm7D"] = futures_candle_df['vol_c2c_'+str(x/24)+"D"].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
    futures_candle_df['vol_c2c_'+str(x/24)+"D"+"_ewm21D"] = futures_candle_df['vol_c2c_'+str(x/24)+"D"].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
    futures_candle_df['vol_park_'+str(x/24)+"D"+"_ewm7D"] = futures_candle_df['vol_park_'+str(x/24)+"D"].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
    futures_candle_df['vol_park_'+str(x/24)+"D"+"_ewm21D"] = futures_candle_df['vol_park_'+str(x/24)+"D"].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
    futures_candle_df['vol_garman_'+str(x/24)+"D"+"_ewm7D"] = futures_candle_df['vol_garman_'+str(x/24)+"D"].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
    futures_candle_df['vol_garman_'+str(x/24)+"D"+"_ewm21D"] = futures_candle_df['vol_garman_'+str(x/24)+"D"].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
    futures_candle_df['vol_rogers_'+str(x/24)+"D"+"_ewm7D"] = futures_candle_df['vol_rogers_'+str(x/24)+"D"].ewm(halflife='7 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
    futures_candle_df['vol_rogers_'+str(x/24)+"D"+"_ewm21D"] = futures_candle_df['vol_rogers_'+str(x/24)+"D"].ewm(halflife='21 days', times=pd.DatetimeIndex(futures_candle_df['dt'])).mean()
#24hr and 1hr changes  
shifts = [1, 24]
for col in futures_candle_df.columns:
    if "vol_" in col:
        for s in shifts:
            shift_df_temp = futures_candle_df.copy(deep=True).shift(s)
            futures_candle_df = create_changePCT(futures_candle_df, shift_df_temp, col, str(s)+"HR")   
 


/tmp/ipykernel_185607/1640958538.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column+"Chg"+time] = (futures_candle_df[column]-shift[column])
/tmp/ipykernel_185607/1640958538.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column+"Chg%"+time] = df[column+"Chg"+time]/shift[column]
/tmp/ipykernel_185607/1640958538.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [8]:
#%%get pred
futures_candle_df['closeChg%_forward24HR'] = futures_candle_df['closeChg%24HR'].shift(-24)
futures_candle_df['closeChg%_forward1HR'] = futures_candle_df['closeChg%1HR'].shift(-1)
#get up down or chop based on sdev
std_24 = futures_candle_df['closeChg%_forward24HR'].std()
std_1 = futures_candle_df['closeChg%_forward1HR'].std()
for index, row in futures_candle_df.iterrows():
    if abs(row['closeChg%_forward24HR'])>=  std_24:
        if row['closeChg%_forward24HR'] > 0:
            futures_candle_df.at[index,'UpDownPred24HR' ] =1
        else:
            futures_candle_df.at[index,'UpDownPred24HR' ] =-1
    else:
        futures_candle_df.at[index,'UpDownPred24HR' ] =0
    if abs(row['closeChg%_forward1HR'])>=  std_1:
        if row['closeChg%_forward1HR'] > 0:
            futures_candle_df.at[index,'UpDownPred1HR' ] =1
        else:
            futures_candle_df.at[index,'UpDownPred1HR' ] =-1
    else:
        futures_candle_df.at[index,'UpDownPred1HR' ] =0



futures_candle_df.dropna(inplace=True)
futures_candle_df.reset_index(drop = True, inplace=True)  

In [9]:
#%% classification 24hr
start_class = datetime.now()
from pycaret.classification import *
#number_columns = 100/(len(futures_candle_df.columns))
exp_clf24 = setup(futures_candle_df,target='UpDownPred24HR',
        ignore_features=['dt', 'exchange', 'timestamp', 'instrument', 'closeChg%_forward1HR', 'closeChg%_forward24HR', 'UpDownPred1HR'],session_id=11,
        profile=False,  use_gpu=True,  normalize = True,  remove_multicollinearity=True) 
end_setup_class= datetime.now()
models_class = compare_models(turbo=True, n_select =4)
model_df_class = pull()
best_models_class = model_df_class.iloc[0:4]

end_class = datetime.now()
print('class_setup_config', (end_setup_class - start_class))

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recomp

,Description,Value
0,Session id,11
1,Target,UpDownPred24HR
2,Target type,Multiclass
3,Target mapping,"-1.0: 0, 0.0: 1, 1.0: 2"
4,Original data shape,"(28729, 684)"
5,Transformed data shape,"(28729, 113)"
6,Transformed train set shape,"(20110, 113)"
7,Transformed test set shape,"(8619, 113)"
8,Ignore features,7
9,Numeric features,676


[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9273,0.9690,0.9273,0.9262,0.9252,0.7935,0.7970,1.0090
rf,Random Forest Classifier,0.9085,0.9635,0.9085,0.9091,0.9026,0.7230,0.7373,1.7280
catboost,CatBoost Classifier,0.9039,0.9574,0.9039,0.9043,0.8974,0.7077,0.7230,10.1190
lightgbm,Light Gradient Boosting Machine,0.9036,0.9586,0.9036,0.9040,0.8969,0.7063,0.7219,1.1970
knn,K Neighbors Classifier,0.8780,0.9162,0.8780,0.8727,0.8730,0.6461,0.6513,0.6470
dt,Decision Tree Classifier,0.8771,0.8333,0.8771,0.8775,0.8772,0.6714,0.6715,3.5070
gbc,Gradient Boosting Classifier,0.8108,0.8489,0.8108,0.8250,0.7575,0.2564,0.3613,141.0750
ridge,Ridge Classifier,0.7790,0.0000,0.7790,0.7317,0.6947,0.0587,0.1280,0.4030
lr,Logistic Regression,0.7787,0.6937,0.7787,0.7191,0.7045,0.0946,0.1560,12.1650
dummy,Dummy Classifier,0.7755,0.5000,0.7755,0.6014,0.6774,0.0000,0.0000,0.4280


class_setup_config 0:00:15.299583


In [11]:
evaluate_model(models_class[0])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [15]:
et = create_model('et')
rf = create_model('rf')
cat = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9289,0.9685,0.9289,0.9277,0.9271,0.7988,0.8015
1,0.9294,0.9750,0.9294,0.9280,0.9276,0.8006,0.8031
2,0.9209,0.9672,0.9209,0.9192,0.9187,0.7750,0.7784
3,0.9329,0.9744,0.9329,0.9317,0.9312,0.8115,0.8137
4,0.9269,0.9704,0.9269,0.9261,0.9244,0.7896,0.7944
5,0.9269,0.9664,0.9269,0.9258,0.9244,0.7905,0.7951
6,0.9249,0.9692,0.9249,0.9234,0.9232,0.7891,0.7913
7,0.9274,0.9663,0.9274,0.9263,0.9251,0.7928,0.7968
8,0.9334,0.9733,0.9334,0.9321,0.9319,0.8144,0.8159


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9100,0.9663,0.9100,0.9098,0.9048,0.7302,0.7420
1,0.9155,0.9704,0.9155,0.9155,0.9107,0.7476,0.7588
2,0.9010,0.9618,0.9010,0.9009,0.8944,0.6987,0.7137
3,0.9150,0.9698,0.9150,0.9142,0.9108,0.7492,0.7580
4,0.9030,0.9596,0.9030,0.9040,0.8961,0.7029,0.7197
5,0.9090,0.9605,0.9090,0.9110,0.9026,0.7218,0.7390
6,0.9110,0.9637,0.9110,0.9111,0.9057,0.7330,0.7454
7,0.9080,0.9631,0.9080,0.9093,0.9016,0.7196,0.7359
8,0.9120,0.9657,0.9120,0.9120,0.9068,0.7367,0.7485


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9080,0.9631,0.9080,0.9078,0.9024,0.7232,0.7358
1,0.8991,0.9609,0.8991,0.8989,0.8919,0.6910,0.7073
2,0.9080,0.9578,0.9080,0.9082,0.9021,0.7215,0.7355
3,0.9095,0.9638,0.9095,0.9094,0.9039,0.7283,0.7407
4,0.9000,0.9562,0.9000,0.9004,0.8929,0.6940,0.7103
5,0.9115,0.9546,0.9115,0.9136,0.9054,0.7304,0.7467
6,0.9080,0.9570,0.9080,0.9069,0.9030,0.7265,0.7369
7,0.9015,0.9521,0.9015,0.9028,0.8943,0.6975,0.7155
8,0.8986,0.9620,0.8986,0.8977,0.8918,0.6926,0.7067


In [17]:
blend1 = blend_models(estimator_list = [et, rf, cat])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9199,0.9718,0.9199,0.9191,0.9165,0.7661,0.7732
1,0.9175,0.9738,0.9175,0.9173,0.9131,0.7550,0.7650
2,0.9135,0.9672,0.9135,0.9130,0.9089,0.7429,0.7529
3,0.9224,0.9738,0.9224,0.9215,0.9192,0.7749,0.7811
4,0.9130,0.9674,0.9130,0.9133,0.9079,0.7394,0.7511
5,0.9234,0.9664,0.9234,0.9242,0.9194,0.7730,0.7833
6,0.9184,0.9675,0.9184,0.9177,0.9147,0.7610,0.7689
7,0.9170,0.9660,0.9170,0.9173,0.9125,0.7531,0.7638
8,0.9180,0.9715,0.9180,0.9167,0.9144,0.7611,0.7678
